In [1]:
from pathlib import Path
import time
import zmq
import json

In [2]:
class StoredDict():
    def __init__(self,bucket='default'):
        self.data = {}
        self.file = Path(bucket+'.json')
        if self.file.exists():
            self.data = json.loads(self.file.read_text())
    def update(self, other):
        self.data.update(other)
        self.file.write_text(json.dumps(self.data,indent=4))
    def delete(self,key):
        if key in self.data:
            del self.data[key]
            self.file.write_text(json.dumps(self.data,indent=4))
    def toPacket(self):
        return self.data
    
    

In [ ]:
context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5555")
data = {
    'default':StoredDict('default')
}
print('loop running')
while True:
    #  Wait for next request from client
    message = socket.recv()    
    if len(message)>1:
        bucket,message  = message.decode('utf-8').split(':',maxsplit=1)
        if bucket in data:
            if message.startswith('del'):
                data[bucket].delete(message[4:])
            elif len(message)>1:
                message = json.loads(message)
                data[bucket].update(message)
        else:
            data[bucket] = StoredDict(bucket)
    
        socket.send_json(data[bucket].toPacket())
    else:
        socket.send_json({k:data[k].toPacket() for k in data})
    print()
